Checking calculations compared to strava rider data for the 2023 TDF

In [35]:
import numpy as np
import pandas as pd
from procyclingstats import Stage
import power_helper as ph
import plotly.express as px
df = pd.read_csv("Data_edited.csv").iloc[:,2:]
df = df[df.year == 2023]

In [36]:
# Get stage data for each person
def to_seconds(time):
    h, m, s = time.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

# Get stages
def get_stages(stage_index, name, df):
    stage = Stage(df.iloc[stage_index].stage_url)
    time = next(item for item in stage.results('rider_name', 'time') if item['rider_name'] == name)['time']
    velocity = (stage.distance()*1000)/to_seconds(time)
    return velocity

def get_velocities(velocities, np, name, df):
    for i, avg in enumerate(np):
        if avg != 0:
            velocities.append(get_stages(i, name, df))
        else:
            velocities.append(0)
    return velocities
    
import power_helper as ph

# 2023 TDF power numbers 
# Victor Campenaerts 72Kg Time trail
# Florian Vermeersch 85Kg One day racer
# JOHANNESSEN Tobias Halland 62kg  CLIMBER
# Victor lafay 65kg CLIMBER
# Egan Bernal 60 KG GC/climber
# Michal Kwiatkowski 68KG GC/climber/TT
# Stage 1: 
Campenaerts_avg = [237, 238, 201, 152, 296, 227, 186, 228, 247, 263, 185, 264, 174, 0, 0, 270, 246, 0, 0, 0, 179]
Campenaerts_NP =  [306, 312, 253, 230, 353, 277, 244, 305, 319, 339, 252, 327, 237, 0, 0, 315, 287, 0, 0, 0, 268]
Johannessen_avg=  [239, 223, 189, 144, 251, 282, 160, 199, 221, 249, 183, 000, 212, 242, 227, 316, 279, 173, 225, 280, 164]
Johannessen_np =  [292, 284, 242, 205, 303, 322, 209, 257, 271, 310, 241, 000, 259, 289, 288, 330, 315, 231, 272, 334, 251]
Lafay_avg      =  [254, 243, 214, 158, 260, 290, 192, 252, 231, 249, 217, 283, 255, 265, 000, 000, 261, 206, 258] #DNF last 2 stages
Lafay_np       =  [308, 308, 275, 218, 306, 258, 249, 306, 271, 315, 273, 337, 290, 303, 000, 000, 302, 253, 303, 000, 000]
Bernal_avg      = [219, 211, 180, 138, 243, 242, 160, 206, 211, 220, 206, 231, 241, 233, 210, 291, 145, 167, 208, 260, 144]
Bernal_np       = [267, 265, 228, 200, 284, 284, 215, 258, 257, 273, 257, 282, 270, 282, 263, 298, 252, 210, 264, 299, 217]
Kwiatowski_avg  = [226, 212, 189, 161, 240, 268, 193, 227, 278, 256, 197, 215, 270, 240, 216, 320, 272, 176, 216, 267, 151]
Kwiatowski_np   = [286, 283, 250, 229, 302, 317, 248, 280, 228, 313, 260, 280, 315, 294, 270, 339, 320, 235, 289, 313, 250]

df['Campenaerts_velocities'] = get_velocities([], Campenaerts_NP, "CAMPENAERTS Victor", df)
df['Campenaerts_NP'] = Campenaerts_NP

df['JOHANNESSEN_velocities'] = get_velocities([], Johannessen_np, "JOHANNESSEN Tobias Halland", df)
df['JOHANNESSEN_NP'] = Johannessen_np

df['Lafay_velocities'] = get_velocities([], Lafay_np, "LAFAY Victor", df)
df['Lafay_NP'] = Lafay_np

df['Bernal_velocities'] = get_velocities([], Bernal_np, "BERNAL Egan", df)
df['Bernal_NP'] = Bernal_np

df['Kwiatkowski_velocities'] = get_velocities([], Kwiatowski_np, "KWIATKOWSKI Michał", df)
df['Kwiatkowski_NP'] = Kwiatowski_np

df_search = df[['stage_distance', 'stage_grade', 'profile_icon', 
                'Campenaerts_velocities', 'Campenaerts_NP', 
                'JOHANNESSEN_velocities', 'JOHANNESSEN_NP', 
                'Lafay_velocities', 'Lafay_NP', 
                'Bernal_velocities', 'Bernal_NP', 
                'Kwiatkowski_velocities', 'Kwiatkowski_NP']]
df_search.to_csv('searched_powers.csv')

In [37]:
df_search = pd.read_csv("searched_powers.csv")

In [38]:
flat =  df_search[(df_search.profile_icon == 'p1')]
hilly = df_search[(df_search.profile_icon == 'p2') | (df_search.profile_icon == 'p3')]
mountains = df_search[(df_search.profile_icon == 'p4') | (df_search.profile_icon == 'p5')]

In [53]:
# 2023 TDF power numbers 
# Victor Campenaerts 72Kg Time trail
# JOHANNESSEN Tobias Halland 62kg  CLIMBER
# Victor lafay 65kg CLIMBER
# Egan Bernal 60 KG GC/climber
# KWIATKOWSKI Michał 68KG GC/climber/TT

def get_powers(slope_mod, velocity_mod, df):
    avg = 0
    for stage in df.itertuples():
        if stage.Campenaerts_velocities != 0.0:
            avg = np.mean((avg, 
                          abs(ph.cycling_power_profile_mods(stage.stage_grade, 72, stage.Campenaerts_velocities, stage.profile_icon, slope_mod, velocity_mod) 
                          - stage.Campenaerts_NP)))
        if stage.JOHANNESSEN_velocities != 0.0:
            avg = np.mean((avg, 
                          abs(ph.cycling_power_profile_mods(stage.stage_grade, 62, stage.JOHANNESSEN_velocities, stage.profile_icon, slope_mod, velocity_mod) 
                          - stage.JOHANNESSEN_NP)))
        if stage.Lafay_velocities != 0.0:
            avg = np.mean((avg, 
                          abs(ph.cycling_power_profile_mods(stage.stage_grade, 65, stage.Lafay_velocities, stage.profile_icon, slope_mod, velocity_mod) 
                          - stage.Lafay_NP)))
        if stage.Bernal_velocities != 0.0:
            avg = np.mean((avg, 
                          abs(ph.cycling_power_profile_mods(stage.stage_grade, 60, stage.Bernal_velocities, stage.profile_icon, slope_mod, velocity_mod) 
                          - stage.Bernal_NP)))
        if stage.Kwiatkowski_velocities != 0.0:
            avg = np.mean((avg, 
                          abs(ph.cycling_power_profile_mods(stage.stage_grade, 68, stage.Kwiatkowski_velocities, stage.profile_icon, slope_mod, velocity_mod) 
                          - stage.Kwiatkowski_NP)))
    return avg
            
        

slope_mods = [0,0]
velocity_mods = [0,0]

best_result = float('+inf')
best_results = []
best_combination = None
best_combinations = []
dfs = [flat, hilly, mountains]

for i, df in enumerate(dfs):
    for slope_mod_uphill in range(5,30, 5):
        for slope_mod_flat in range(5,30, 5):
            for velocity_mod_flat in range(5,30, 5):
                for velocity_mod_uphill in range(5,30, 5):
                    current_result = get_powers([slope_mod_flat/10, slope_mod_uphill/10], 
                                                [velocity_mod_flat/10, velocity_mod_uphill/10], 
                                                df)
                    if current_result < best_result:
                        best_result = current_result
                        best_combination = ([slope_mod_flat/10, slope_mod_uphill/10], [velocity_mod_flat/10, velocity_mod_uphill/10])
                        
    print(best_result)
    best_results.append(best_result)
    best_combinations.append(best_combination)

print("Best Combination:", best_combinations[:])
print("Best Result:", best_results[:])


21.020979575255552
12.931007194970796
9.677467332558363
Best Combination: [([0.5, 0.5], [1.0, 1.0]), ([0.5, 1.0], [0.5, 1.0]), ([1.5, 1.5], [0.5, 0.5])]
Best Result: [21.020979575255552, 12.931007194970796, 9.677467332558363]


In [91]:
def get_powers(slope_mod, velocity_mod, df):
    estimates = []
    actuals = []
    for stage in df.itertuples():
        if stage.Campenaerts_velocities != 0.0:
            estimates.append(ph.cycling_power_profile_mods(stage.stage_grade, 72, stage.Campenaerts_velocities, stage.profile_icon, slope_mod, velocity_mod))
            actuals.append(stage.Campenaerts_NP)
        if stage.JOHANNESSEN_velocities != 0.0:
            estimates.append(ph.cycling_power_profile_mods(stage.stage_grade, 62, stage.JOHANNESSEN_velocities, stage.profile_icon, slope_mod, velocity_mod))
            actuals.append(stage.JOHANNESSEN_NP)
        if stage.Lafay_velocities != 0.0:
            estimates.append(ph.cycling_power_profile_mods(stage.stage_grade, 65, stage.Lafay_velocities, stage.profile_icon, slope_mod, velocity_mod))
            actuals.append(stage.Lafay_NP)
        if stage.Bernal_velocities != 0.0:
            estimates.append(ph.cycling_power_profile_mods(stage.stage_grade, 60, stage.Bernal_velocities, stage.profile_icon, slope_mod, velocity_mod))
            actuals.append(stage.Bernal_NP)
        if stage.Kwiatkowski_velocities != 0.0:
            estimates.append(ph.cycling_power_profile_mods(stage.stage_grade, 68, stage.Kwiatkowski_velocities, stage.profile_icon, slope_mod, velocity_mod))
            actuals.append(stage.Kwiatkowski_NP)
            
    return estimates, actuals
            
flat_estimates, flat_actuals = get_powers([0.5, 0.5], [1.0, 1.0], flat)
hilly_estimates, hilly_actuals = get_powers([0.5, 1.0], [0.5, 1.0], hilly)
mountain_estimates, mountain_actuals = get_powers([1.5, 1.5], [0.5, 0.5], mountains)
estimates = flat_estimates + hilly_estimates + mountain_estimates
actuals = flat_actuals + hilly_actuals + mountain_actuals

In [93]:

fig = px.scatter(y=[estimates, actuals], range_y=[0,400])
fig.data[0].name = "estimate"
fig.data[1].name = "actual"
fig.show()